In [ ]:
from __future__ import print_function
import os, sys
from pyspark import SparkContext,SparkConf
from operator import add
import itertools
import copy
from operator import itemgetter
from collections import Counter
from itertools import combinations
from pyspark.sql import SQLContext
from pyspark.sql.functions import lower
from pyspark.sql.functions import *
import jellyfish
import pandas as pd
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql import SparkSession
import pickle

In [ ]:
spark = SparkSession.builder.appName("yelp").getOrCreate()

In [ ]:
conf = SparkConf().setAppName("App")
conf = (conf.setMaster('local[*]').set('spark.executor.memory', '4G').set('spark.driver.memory', '45G').set('spark.driver.maxResultSize', '10G'))
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [ ]:
df_test_reviews = sc.textFile('ParthTrain.csv')

In [ ]:
df_test_reviews.take(5)

In [ ]:
train_data_reviews = df_test_reviews.map(lambda x: x.split(','))

In [ ]:
train_data_reviews.take(5)

In [ ]:
df_train_data_reviews = train_data_reviews.toDF(['business_id','stars','user_id'])

In [ ]:
distinct_business = df_train_data_reviews.select("business_id").distinct().toPandas()

In [ ]:
distinct_business.insert(0, 'ID', range(0, 0 + len(distinct_business)))

In [ ]:
business_id_dictionary = {}
id_business_dictionary = {}
for index,row in distinct_business.iterrows():
    if row["ID"] not in id_business_dictionary:
        id_business_dictionary[row["ID"]] = row["business_id"]
        business_id_dictionary[row["business_id"]] = row["ID"]

In [ ]:
distinct_users = df_train_data_reviews.select("user_id").distinct().toPandas()

In [ ]:
distinct_users.insert(0, 'ID', range(0, 0 + len(distinct_users)))

In [ ]:
type(distinct_users)

In [ ]:
user_id_dictionary = {}
id_user_dictionary = {}
for index,row in distinct_users.iterrows():
    if (row["ID"] == "hDamkEc4NvaXDMD1CMtG2A"):
        print("ok")
    if row["ID"] not in distinct_users:
        id_user_dictionary[row["ID"]] = row["user_id"]
        user_id_dictionary[row["user_id"]] = row["ID"]

In [ ]:
distinct_users.to_csv("user-id-mapping.csv", encoding='utf-8', index=False)

In [ ]:
distinct_business.to_csv("business-id-mapping", encoding='utf-8', index=False)

In [ ]:
udf = UserDefinedFunction(lambda x: str(x.replace(x,user_id_dictionary[x])))

In [ ]:
pd_df_train_data_reviews = df_train_data_reviews.toPandas()

In [ ]:
pd_df_train_data_reviews

In [ ]:
# temp1.replace({"business_id": business_id_dictionary})

In [ ]:
# a = temp1.copy()

In [ ]:
pd_df_train_data_reviews["business_id"] = pd_df_train_data_reviews['business_id'].map(business_id_dictionary).fillna(pd_df_train_data_reviews['business_id'])

In [ ]:
pd_df_train_data_reviews["user_id"] = pd_df_train_data_reviews['user_id'].map(user_id_dictionary).fillna(pd_df_train_data_reviews['user_id'])

In [ ]:
pd_df_train_data_reviews

In [ ]:
pd_df_train_data_reviews["stars"] = pd_df_train_data_reviews.stars.astype(int)

In [ ]:
pd_df_train_data_reviews.info()

In [ ]:
df_pd_df_train_data_reviews = sqlContext.createDataFrame(pd_df_train_data_reviews)

In [ ]:
df_pd_df_train_data_reviews

In [ ]:
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="business_id", ratingCol="stars",
          coldStartStrategy="drop")

In [ ]:
model1 = als.fit(df_pd_df_train_data_reviews)

# test data

In [ ]:
df_test_data = sc.textFile('subset_cross.csv')
header = df_test_data.first()
df_test_data = df_test_data.filter(lambda line: line != header)

In [ ]:
df_test_data.take(5)

In [ ]:
test_data_ratings = df_test_data.map(lambda x: x.split(','))

In [ ]:
test_data_ratings.take(5)

In [ ]:
df_test_data_header = test_data_ratings.toDF(['ID','business_id','user_id']).toPandas()

In [ ]:
df_test_data_header = df_test_data_header.drop(['ID'], axis=1)

In [ ]:
unique_test_business = df_test_data_header.business_id.unique()
unique_test_user = df_test_data_header.user_id.unique()

In [ ]:
id_business_dictionary

In [ ]:
for business in unique_test_business:
    if business not in business_id_dictionary:
        temp = len(business_id_dictionary)
        business_id_dictionary[business] = temp
        id_business_dictionary[temp] = business
        print("a")

In [ ]:
for user in unique_test_user:
    if user not in user_id_dictionary:
        temp = len(user_id_dictionary)
        user_id_dictionary[user] = temp
        id_user_dictionary[temp] = user
        print("b")

In [ ]:
df_test_data_header["business_id"] = df_test_data_header['business_id'].map(business_id_dictionary).fillna(df_test_data_header['business_id'])

In [ ]:
df_test_data_header["user_id"] = df_test_data_header['user_id'].map(user_id_dictionary).fillna(df_test_data_header['user_id'])

In [ ]:
test_data_final = sqlContext.createDataFrame(df_test_data_header)

In [ ]:
predictions = model1.transform(test_data_final)

In [ ]:
movieRecs = model1.recommendForAllItems(10)

In [ ]:
user_rec = movieRecs.collect()

In [ ]:
user_rec[0]

In [ ]:
movier = model1.recommendForAllItems(100)

In [ ]:
userr = movier.collect()

In [ ]:
shaily_dictionary = {}

In [ ]:
for item in userr:
    businessID = item[0]
    businessIDString = id_business_dictionary[businessID]
#     print(businessIDString)
    shaily_dictionary[businessIDString] = {}
    for i in range(0,len(item[1])):
        userIDString = id_user_dictionary[item[1][i][0]]
        shaily_dictionary[businessIDString][userIDString] = item[1][i][1]
#         print(item[1][i])

In [ ]:
shaily_dictionary

In [ ]:
with open('ratingDictionary.pickle', 'wb') as handle:
    pickle.dump(shaily_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
tempuser = predictions.collect()

In [ ]:
shaily_parikh_dictionary = {}

In [ ]:
tempuser

In [ ]:
for item in tempuser:
    businessID = item["business_id"]
    businessIDString = id_business_dictionary[businessID]
    if businessIDString not in shaily_parikh_dictionary:
        shaily_parikh_dictionary[businessIDString] = {}
        userIDString = id_user_dictionary[item["user_id"]]
        shaily_parikh_dictionary[businessIDString][userIDString] = item["prediction"]
    elif businessIDString in shaily_parikh_dictionary:
        userIDString = id_user_dictionary[item["user_id"]]
        shaily_parikh_dictionary[businessIDString][userIDString] = item["prediction"]
        
#     print(businessIDString)

In [ ]:
shaily_parikh_dictionary

In [ ]:
with open('ratingDictionary1.pickle', 'wb') as handle:
    pickle.dump(shaily_parikh_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# distinct_users.loc[distinct_users["user_id"] == "hDamkEc4NvaXDMD1CMtG2A"]

In [ ]:
# distinct_users

In [ ]:
# len(user_id_dictionary)

In [ ]:
# distinct_users.count()